## Photometry on MIRI images

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
import pandas as pd
import warnings
import subprocess
import miri_utils.photometry as phot

from astropy.io import fits
from astropy.wcs import FITSFixedWarning
from astropy.table import Table

warnings.simplefilter("ignore", category=FITSFixedWarning)


cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"


# Section to obtain modified apertures

Let's inspect Amirs table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
#print(table[:5])
table.info()
print(table.columns)

Now let's try and call the function:

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

# Get all FITS file paths
fits_files = glob.glob(os.path.join(cutout_dir, '*.fits'))

# Get the basenames of the FITS files
fits_fnames = [os.path.basename(f) for f in fits_files]

adjusted_apertures = []

for fname in fits_fnames:
    id = fname.split('_')[0]
    filter = fname.split('_')[1]
    if filter == 'F1800W': continue
    survey_obs = fname.split('_')[3]
    if '003' in survey_obs:
        survey = 'primer'
        obs = '003'
    elif '004' in survey_obs:
        survey = 'primer'
        obs = '004'
    elif '1' in survey_obs:
        survey = 'cweb'
        obs = '1'
    elif '2' in survey_obs:
        survey = 'cweb'
        obs = '2'
    else:
        print(f"Unknown survey and/or observation number for galaxy {id}:\n")
        print(survey_obs)
    
    # Call and collect results
    result = phot.adjust_aperture(id, filter, survey, obs, phot_dir, save_plot=False)
    if result:
        adjusted_apertures.append(result)

# After loop: create a DataFrame
df_apertures = pd.DataFrame(adjusted_apertures)

df_path = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_v2.csv'

# (optional) Save to CSV or integrate into photometry table
df_apertures.to_csv(df_path, index=False)

Now we can easily open any given FITS file with its corresponding ellipse region

In [ ]:
# --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
region_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

id = '10245'
filter = 'F770W'
survey_obs = 'primer004'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
subprocess.run(["ds9", cutout_path, "-regions", reg_path])


Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
print(table['Image_Err'].shape)

# This is where we do most of the work

# Section to perform the actual photometry

First, let's set some parameters straight:

pixscale_arcsec = 0.11092  # arcsec per pixel

pix_area_sr = 2.89208962133982e-13  # from MIRI header


In case the code should be tested

In [ ]:
test_ids = ['8465', '7922', '9871', '12202', '8843', '7904', '8338', '10021', '10245', '11136', '12340', '20397']

In [ ]:
# --- Parameters ---
cutouts_folder = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
output_folder = '/home/bpc/University/master/Red_Cardinal/photometry/'
aperture_table = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_v2.csv'
fig_path = '/home/bpc/University/master/Red_Cardinal/photometry/Plots_MIRI_phot_v2/'
os.makedirs(output_folder, exist_ok=True)

# Get all possible F770W files
all_f770w_files = glob.glob(os.path.join(cutouts_folder, f'*F770W*.fits'))

# Group F770W files by galaxy ID and filter
f770w_files = []
galaxy_ids = set([os.path.basename(f).split('_')[0] for f in all_f770w_files])

for galaxy_id in galaxy_ids:
    # Find all F770W files for this galaxy ID
    matching_files = [f for f in all_f770w_files if os.path.basename(f).startswith(galaxy_id)]
    
    # Handle special case for galaxy 11853
    if galaxy_id == '11853':
        # Use the cweb2 file if available
        cweb2_files = [f for f in matching_files if 'cweb2' in f.lower()]
        if cweb2_files:
            f770w_files.append(cweb2_files[0])
            continue  # Skip to the next galaxy
    
    # Prioritise PRIMER over COSMOS-Web
    primer_files = [f for f in matching_files if 'primer' in f.lower()]
    cweb_files = [f for f in matching_files if 'cweb' in f.lower()]
    
    if primer_files:
        f770w_files.append(primer_files[0])  # Prefer PRIMER file
    elif cweb_files:
        f770w_files.append(cweb_files[0])  # Use CWEB only if no PRIMER available

# Get all F1800W files
f1800w_files = glob.glob(os.path.join(cutouts_folder, f'*F1800W*.fits'))

psf_f770w = phot.get_psf('F770W')
psf_f1800w = phot.get_psf('F1800W')

phot.perform_photometry(f770w_files, aperture_table, output_folder, 
                        psf_f770w, fig_path=fig_path, suffix='_v2', 
                        sigma=2.0, annulus_factor=3.0
)
phot.perform_photometry(f1800w_files, aperture_table, output_folder, 
                        psf_f1800w, fig_path=fig_path, suffix='_v2', 
                        sigma=2.0, annulus_factor=3.0
)
phot.combine_figures(fig_path)

Create the FITS table

In [ ]:
# Your existing code to save CSV files for each filter
results_folder = '/home/bpc/University/master/Red_Cardinal/photometry/results/'
os.makedirs(results_folder, exist_ok=True)

# Now create the combined FITS table
phot.combine_filter_csv_to_fits(results_folder)

Collect galaxy IDs and their corresponding observations:

In [ ]:
NIRCam_table =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
table = Table.read(NIRCam_table)
table.info()
f444w_ids = list(table['ID'])

MIRI_table = '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI.fits'
table = Table.read(MIRI_table)
table.info()
f770w_ids = list(table['ID'])

cutout_dir = '/home/bpc/University/master/Red_Cardinal/cutouts_phot/'
f1800w_ids = glob.glob(os.path.join(cutout_dir, '*F1800W*.fits'))
for i, f in enumerate(f1800w_ids):
    f = os.path.basename(f).split('_')[0]
    f1800w_ids[i] = f

# Convert the arrays to sets for efficient comparison
set_f444w = set(f444w_ids)
set_f770w = set(f770w_ids)
set_f1800w = set(f1800w_ids)

# 1) IDs in all 3 arrays
in_all_three = set_f444w & set_f770w & set_f1800w

# 2) IDs in f444w_ids and f770w_ids only (not in f1800w_ids)
in_f444w_and_f770w_only = (set_f444w & set_f770w) - set_f1800w

# 3) IDs only in f444w_ids and not in the other 2
only_in_f444w = set_f444w - set_f770w - set_f1800w

# Convert back to sorted lists if needed
in_all_three = sorted(list(in_all_three))
in_f444w_and_f770w_only = sorted(list(in_f444w_and_f770w_only))
only_in_f444w = sorted(list(only_in_f444w))

print(f"{len(in_all_three)} IDs in all three arrays: {in_all_three}")
print(f"{len(in_f444w_and_f770w_only)} IDs in f444w_ids and f770w_ids only: {in_f444w_and_f770w_only}")
print(f"{len(only_in_f444w)} IDs only in f444w_ids: {only_in_f444w}")



Check the contents of the original FITS files:

In [ ]:
fname = './../cutouts/7102_F770W_cutout_cweb1.fits'
#with fits.open(fname) as hdul:
    #hdul.info()


fname = './../MIRI/PRIMER_003/jw01837-o003_t003_miri_f770w_i2d.fits'
with fits.open(fname) as hdul:
    hdul.info()

fname = './../MIRI_shifted/PRIMER_003_shifted/jw01837-o003_t003_miri_f770w_i2d_shifted.fits'
with fits.open(fname) as hdul:
    hdul.info()

